# 02 - Creation de merge clean metrics dataset 

Ce notebook génère :

- 1 fichier csv "merge_clean_metrics_dataset.csv"

Etapes de nettoyage :

- Suppression des colonnes n'ayant que des valeurs nulles

- Suppression des colonnes avec informations redondantes (identification=message_events) ou inutiles (id de message)

- Conversion des types de colonnes avec le type de valeurs

- Remplacement des valeurs nulles

- Encodage des codes d'identification en chaine de caractères (maj du metrics_events_dict.json) et de la criticité

# A. Imports

## Librairies

In [ ]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path

## Données

In [ ]:
# source path to raw metrics dataset
source_csv = '../data/metrics/raw_merge_metrics_dataset.csv'
# target path to save metrics dictionnaire
save_json ='../data/metrics/metrics_events_dict.json'
# target path to save merge raw metrics dataset
save_csv = '../data/metrics/clean_merge_metrics_dataset.csv'

# B. Dataframe

## a) Import dataset

In [ ]:
# création d'un dataframe à partir du csv de données
df = pd.read_csv(Path(source_csv), index_col=0)
# réindexation à 0
df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df.info()

## b) Selection des colonnes

In [ ]:
# suppression des colonnes ne contenant que des valeurs nulles
df = df.dropna(axis=1, how='all')

In [ ]:
# on supprime les colonnes doublons (message=identification)
df = df.drop(['id', 'message_events'], axis=1)

In [ ]:
# on converti les float en entier 64
df.varnishLevelsTargetvolume = pd.to_numeric(df.varnishLevelsTargetvolume).astype('int64')

In [ ]:
df.info()

### 1) Colonne 'timestamp'

In [ ]:
# on renomme la colonne timestamp_events
df = df.rename(columns={'timestamp_events':'timestamp'})
# on remplace des valeurs maquantes de timestamp par celle de created_at
df.timestamp = df.timestamp.fillna(df['created_at'])
# on converti les valeur en datetim
df.timestamp = pd.to_datetime(df.timestamp, utc=True)
# on supprime la colonne doublon (created_at=tiemstamp)
df = df.drop(['created_at'], axis=1)

### 2) Colonne 'identification'

In [ ]:
df.identification_events.unique()

In [ ]:
# on remplace les valeurs nulles par des 0
df.identification_events = df.identification_events.replace(np.nan, 0)

In [ ]:
# on encode les valeurs du type 'str' avec un code
events_id = []
str_code_dict = {}
str_code = 1000
for id in list(df['identification_events'].unique()) :
    try:
        events_id.append(int(id))
    except ValueError:
        str_code_dict[id] = str_code
        events_id.append(str_code)
        str_code += 1
str_code_dict

In [ ]:
# on sauvegarde l'encodage dans metrics_events_dict
inv_str_code_dict = {v: k for k, v in str_code_dict.items()}
with open(file=Path(save_json), mode="r+", encoding='utf-8') as jsonFile:
    data = json.load(jsonFile)
    data['identification encoded'] = inv_str_code_dict
    jsonFile.seek(0)
    json.dump(data, jsonFile, indent=4, ensure_ascii=False)
    jsonFile.close()

In [ ]:
# on remplace dans le dataframe les valeurs du type 'str' avec un code
df.identification_events = df.identification_events.replace(str_code_dict)
# on converti toutes les valeurs en entier
df.identification_events = pd.to_numeric(df.identification_events).astype('int64')

In [ ]:
df.identification_events.unique()

### 3) Encodage des labels 'criticality'

In [ ]:
# on remplace dans le dataframe les valeurs du type 'str' avec un code
df.criticality_events = df.criticality_events.fillna("UNDEFINED")
criticality = {'UNDEFINED': 0, 'INFO': 1, 'WARNING': 2, 'ERROR':3}
df.criticality_events.replace(criticality, inplace=True)
df.criticality_events = pd.to_numeric(df.criticality_events).astype('int64')

In [ ]:
# on sauvegarde l'encodage dans metrics_events_dict
inv_criticality = {v: k for k, v in criticality.items()}
with open(file=Path(save_json), mode="r+", encoding='utf-8') as jsonFile:
    data = json.load(jsonFile)
    data['criticality encoded'] = inv_criticality
    jsonFile.seek(0)
    json.dump(data, jsonFile, indent=4, ensure_ascii=False)
    jsonFile.close()

## c) Output csv

In [ ]:
# sauvegarde du dataframe avant encodage
df.to_csv(path_or_buf=Path(save_csv))

In [ ]:
df.head(3)